# Projeto 1 - Ciência dos Dados

Nome: Ykaro de Sousa Andrade

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
# Obtenção de tweets

## Preparando o ambiente no jupyter:

In [4]:
%%capture

#Instalando o tweepy
!pip install tweepy

In [5]:
import tweepy
import math
import os.path
import pandas as pd
import json
from random import shuffle

___
## Autenticando no  Twitter

* Conta: ***[Preencha aqui o id da sua conta. Ex: @Rodrigo12445122 ]***

In [6]:
#Dados de autenticação do twitter:

#Coloque aqui o identificador da conta no twitter: @Rodrigo12445122
#leitura do arquivo no formato JSON
with open('auth.pass') as fp:    
    data = json.load(fp)

#Configurando a biblioteca. Não modificar
auth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])
auth.set_access_token(data['access_token'], data['access_token_secret'])

___
## Etapas para construção da base de dados:

### Escolha de um produto e coleta das mensagens


In [7]:
#Produto escolhido:
produto = 'puma'

#Quantidade mínima de mensagens capturadas:
n = 500
#Quantidade mínima de mensagens para a base de treinamento:
t = 300 #relevantes

#Filtro de língua, escolha uma na tabela ISO 639-1.
lang = 'pt'

Capturando os dados do twitter:

In [8]:
#Cria um objeto para a captura
#Cria um objeto para a captura
api = tweepy.API(auth)

#Inicia a captura, para mais detalhes: ver a documentação do tweepy
i = 1
msgs = []
for msg in tweepy.Cursor(api.search, q=produto, lang=lang, tweet_mode="extended").items():

    ######################################################
    #Tendo recuperar full_text de retweet
    try:
        msg.retweeted_status.full_text.lower()
    except AttributeError:  # Not a Retweet
        msgs.append(msg.full_text.lower())
    ######################################################

    i += 1

    #Alterando um pouco para forçar a pegar 500
    #if i > n:
    temporario=list(set(list(msgs)))
    if len(temporario) >= 500:
        break

#Embaralhando as mensagens para reduzir um possível viés
shuffle(msgs)
len(msgs)

502

In [9]:
msgs=list(set(list(msgs)))

Salvando os dados em uma planilha Excel:

In [10]:
#Verifica se o arquivo não existe para não substituir um conjunto pronto
if not os.path.isfile('./{0}.xlsx'.format(produto)):
    
    #Abre o arquivo para escrita
    writer = pd.ExcelWriter('{0}.xlsx'.format(produto))

    #divide o conjunto de mensagens em duas planilhas
    dft = pd.DataFrame({'Treinamento' : pd.Series(msgs[:t])})
    dft.to_excel(excel_writer = writer, sheet_name = 'Treinamento', index = False)

    dfc = pd.DataFrame({'Teste' : pd.Series(msgs[t:])})
    dfc.to_excel(excel_writer = writer, sheet_name = 'Teste', index = False)

    #fecha o arquivo
    writer.save()

___
### Classificando as mensagens na coragem

Esta etapa é manual. Faça a mesma pelo Excel.

**Importante: Caso classifique um percentual pequeno de tweets relevantes ou de não relevantes, deve voltar a este notebook e coletar mais tweets diferentes do produto escolhido pelo grupo.**

In [11]:
planilha = pd.read_excel('puma.xlsx')
planilha

,Treinamento,Irrelevante 0 / Relevante 1
0,o neymar inovou a puma https://t.co/qtcuvd1svj,1
1,"a @puma tá deitando nas chuteiras do neymar, m...",1
2,fico assim no da puma affss https://t.co/1ygyh...,0
3,"@g_estrella__ puma tá aí , mas três listra tá ...",1
4,@0ketlyn_s eu acho que um puma ^^,0
...,...,...
295,@brunabud_ aí poderia ter mais que so um puma ...,0
296,to apaixonada num tênis puma mas vi uns coment...,0
297,@leoboschiroli tô quase pedindo aquele puma,1
298,"oq é aquele puma que o neymar largou no story,...",1


In [12]:
planilha['Irrelevante 0 / Relevante 1'].value_counts(True)

0    0.56
1    0.44
Name: Irrelevante 0 / Relevante 1, dtype: float64

In [13]:
planilha_teste = pd.read_excel('puma.xlsx', sheet_name = 'Teste')
planilha_teste['Irrelevante 0 / Relevante 1'].value_counts(True)
planilha_teste

,Teste,Irrelevante 0 / Relevante 1
0,@0chavex0 @ornate_puma vdd to resfriado,0
1,📽 neymar e puma 👀❤!!\ntemos novidades vindo aí...,1
2,@brgmsch mas a puma nem patrocina a aston,0
3,📽 puma neymar jr creativity a inovadora chute...,1
4,"a puma tá a evoluir muito, quem me dera que o ...",1
...,...,...
195,eu e o maninho @thiagobomfim85 . comemoração d...,0
196,@puma_parda hahahahaha! sim! e projetando um s...,0
197,na real eu quero só mais um mv com conceito ig...,0
198,"linda é essa linha da puma pro neymar, você é ...",1


In [14]:
planilha_teste['Irrelevante 0 / Relevante 1'].value_counts(True)

0    0.59
1    0.41
Name: Irrelevante 0 / Relevante 1, dtype: float64